In [1]:
import torch

In [2]:
ratings = torch.tensor([
    [0, 0, 9, 1],
    [3, 0, 7, 0],
    [5, 0, 0, 10],
    [0, 2, 0, 0],
])

In [70]:
# user item rating mat
uir_mat = torch.tensor([
    [1, 3, 9],
    [1, 4, 1],
    [2, 1, 3],
    [2, 3, 7],
    [3, 1, 5],
    [3, 4, 10],
    [4, 2, 2]
])

# Index user and item as 0
uir_mat[:, :2] -= 1

In [88]:
U = torch.tensor([
    [0.8, 1.2, -0.2],
    [2, 1.8, 0.4],
    [0.8, 3, 0.1],
    [1, 0.8, 2.4],
], requires_grad=True)

Vt = torch.tensor([
    [-1.8, 1, -0.2, 1],
    [0.5, 1.2, 0.1, 5],
    [1.4, 4, 0.14, 2],
], requires_grad=True)

In [89]:
# Compute first step
n_iters = 1

expected_list = uir_mat[:, 2]
for i in range(n_iters):
    preds_mat = U@Vt
    pred_list = preds_mat[uir_mat[:, 0], uir_mat[:, 1]]
    
    sq_dif = (expected_list - pred_list)**2
    print(sq_dif)
#     err = 1/len(uir_mat)*torch.sum(sq_dif)
    
    sq_dif[0].backward() # Calculating only respecto to first observation
    
#     print(U.grad)
#     print()
#     print(Vt.grad)
    with torch.no_grad():
        U -= 0.1*U.grad
        Vt -= 0.1*Vt.grad

    print(U)
    print()
    print(Vt)
    U.grad.zero_()
    Vt.grad.zero_()

tensor([82.2286, 29.1600, 26.4196, 51.3229, 23.0400, 36.0000, 91.3936],
       grad_fn=<PowBackward0>)
tensor([[0.4373, 1.3814, 0.0539],
        [2.0000, 1.8000, 0.4000],
        [0.8000, 3.0000, 0.1000],
        [1.0000, 0.8000, 2.4000]], requires_grad=True)

tensor([[-1.8000,  1.0000,  1.2509,  1.0000],
        [ 0.5000,  1.2000,  2.2763,  5.0000],
        [ 1.4000,  4.0000, -0.2227,  2.0000]], requires_grad=True)


Compare results to https://medium.datadriveninvestor.com/how-funk-singular-value-decomposition-algorithm-work-in-recommendation-engines-36f2fbf62cac

NOTE: the estimation for V should not use the updated values for U as the post says

## Test Gradient with indexing

In [55]:
A = torch.rand(3, 3, requires_grad=True)
B = torch.rand(3, 1)
C = torch.rand(3, 1)

print(A)

tensor([[0.7034, 0.3828, 0.2635],
        [0.8279, 0.7125, 0.0677],
        [0.5234, 0.8488, 0.0746]], requires_grad=True)


In [56]:
indexes = torch.tensor([0, 2])

In [57]:
loss = 1/3*torch.sum((C[indexes] - (A@B)[indexes])**2)
print(loss)

loss.backward()

tensor(0.1433, grad_fn=<MulBackward0>)


In [58]:
A.grad

tensor([[-0.0115, -0.0312, -0.0670],
        [ 0.0000,  0.0000,  0.0000],
        [-0.0175, -0.0477, -0.1023]])

Norms

In [93]:
import torch.nn as nn
criterion = nn.MSELoss()

In [98]:
arr1 = torch.tensor([1, 2, 3, 4])
arr2 = torch.tensor([0.5, 1.1, 1.9, 3.5])
N = arr1.shape[0]

print(criterion(arr1, arr2))

rmse = 1/N*torch.sum((arr1 - arr2)**2)
print(rmse)

tensor(0.6300)
tensor(0.6300)


In [100]:
mat = torch.tensor([
    [1, 2],
    [3, 4]
])

print(torch.sum(mat**2))

expected = 1**2 + 2**2 + 3**2 + 4**2
print(expected)

tensor(30)
30
